In [1]:
import pandas as pd

df = pd.read_csv(r'C:\Users\aacjp\Spencer\datasets\MockHyros.csv').drop(['Unnamed: 0'], axis='columns')

The Hack: http://sangaline.com/post/advanced-web-scraping-tutorial/

In [2]:
df.head()

,lead,closed,monthly traffic spend,n paid traffic platforms,domain,business model,monthly revenue,% paid traffic,average order value,new customers,...,ad revenue,roas,cac,profit,keyword matchtype,n links lp,n words lp,trigger words lp,lpc,profit margin
0,FrankKern.com,1,52184.0,5.0,entrepreneurship,info,NaN,9.81,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ImpactTheory.Com,1,NaN,NaN,self-development,media,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,digitalmillionairesecrets.com,1,NaN,NaN,entrepreneurship,info,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mastermind.com,1,NaN,NaN,entrepreneurship,software,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ShopaNova.com,1,NaN,NaN,ecommerce,software,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df['lead'][0]

'FrankKern.com'

https://platform.fullcontact.com/developers/api-keys

In [1]:
import mysql.connector

In [2]:
from getpass import getpass
from mysql.connector import connect, Error

connection = connect(host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "), 
        database='dummy')

Enter username: root
Enter password: ········


In [3]:
cursor = connection.cursor()

In [4]:
cursor

In [7]:
q = 'select * from personas;'
cursor.execute(q)
cursor.fetchall()

[(1, 'psycology', 'El Segundo', 'Toyota Rav 4', 1000, 216000),
 (2, 'marketing', 'Century City', 'BMW i8', 15000, 2400000),
 (3, 'entertainment', 'Brentwood', 'Mercedez-Benz 550', 4500, 408000),
 (7, 'finance', 'Pasadena', 'Honda Accord', 1000, 81000),
 (8, 'AI', 'Burbank', 'Toyota Tacoma', 10000, 9600000),
 (9, 'AI', 'Century City', 'Toyota 86', 30000, 9600000),
 (10, 'retail', 'Glendale', 'Toyota Carolla', 780, 31200),
 (11, 'ecommerce', 'Beverly Hills', 'Ferarri 458', 100000, 12720000),
 (12, 'sales', 'Santa Monica', 'no car', 2000, 108000),
 (13, 'agriculture', 'Topanga', 'Nissan Leaf', 1700, 360000)]

In [6]:
q = '''insert into personas (industry, residence, car, rent, income) 
values ('agriculture', 'Topanga', 'Nissan Leaf', 1700, 360000)'''
cursor.execute(q)

In [8]:
connection.commit()